<img src="mioti.png" style="height: 100px">
<center style="color:#888">Módulo Data Science in IoT<br/>Asignatura Deep Learning</center>

# Worksheet S6: Redes Neuronales Recurrentes. Clasificación de secuencias

## Objetivos 

El objetivo de este worksheet es trabajar las particularidades de las redes neuronales profundas y de su implementación. Más concretamente, hoy veremos las redes neuronales recurrentes de tipo LSTM y su implementación en Keras.

Tras realizar el worksheet habrás aprendido:
- Cómo implementar un modelo LSTM para clasificación de secuencias.
- Cómo reducir el sobreentremiento de este modelo LSTM mediante el uso de dropout
- Cómo enfrentarse a un problema de procesamiento natural de lenguaje.


### Introducción

La clasificación de secuencias consiste en predecir a qué categoría pertenece una secuencia que tomamos como entrada, donde una secuencia es una concatenación ordenada, típicamente en el espacio o en el tiempo.

Este problema es difícil desde un punto de vista técnico, ya que las secuencias pueden variar en longitud y pueden contener un vocabulario muy grande de símbolos de entrada. Sin embargo, lo que hace este problema especialmente interesante es que para obtener un buen rendimiento el modelo necesita aprender y comprender dependencias entre las entradas, es decir, necesita extraer información del contexto.

En este worksheet vamos a implementar una red neuronal LSTM para la clasificación de secuencias en Keras.


### Problema a resolver: IMDB Sentiment Analysis

Para empezar a trabajar con las redes neuronales el problema elegido es el conocido como: IMDB movie review sentiment classification problem, que podriamos traducir como clasificación de opiniones de películas IMDB, donde cada opinión es una secuencia de palabras que forman una opinión de un usuario acerca de una película.

Antes de empezar a resolver el problema, necesitamos estudiar una cosa más: ¿Cómo vamos a introducir el texto en la red neuronal?


### Encoding 

Como ya sabemos, nuestros modelos no pueden recibir como entrada texto y vamos a necesitar codificar de alguna forma nuestros datos a valores numéricos que nuestra red pueda entender. Hay distintas formas de hacer esto, vamos a echar un vistazo rápido a las técnicas que se suelen utilizar.

En esta base de datos podemos encontrarnos con opiniones como las siguientes:

- I thought the movie was going to be bad, but it was actually amazing!

- I thought the movie was going to be amazing, but it was actually bad!

A pesar de ser muy similares, tienen significados diferentes. Esto se debe a que el orden de las palabras es muy importante para entender el significado, por eso las redes necesitan comprender el contexto.

#### Bag of Words

El primer método y el más sencillo es lo que llamamos bag of words o bolsa de palabras. Consiste en en codificar cada palabra con un entero y crear una "bolsa" de palabras que nos dice la frecuencia de cada palabra en una frase o fragmento.

In [1]:
vocabulary = {}  # diccionario para mapear las palabras a enteros
word_encoding = 1
def bag_of_words(text):
  global word_encoding

  words = text.lower().split(" ")  # crea una lista con todas las palabras en el texto
  bag = {}  # diccionario para guardar los encodings y su frecuencia

  for word in words:
    if word in vocabulary:
      encoding = vocabulary[word]  # extrae el encoding del vocabulario
    else:
      vocabulary[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1
    
    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1
  
  return bag

text = "test sentence to see if the algorithm works as a test see see"
bag = bag_of_words(text)
print(bag)
print(vocabulary)

{1: 2, 2: 1, 3: 1, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1}
{'test': 1, 'sentence': 2, 'to': 3, 'see': 4, 'if': 5, 'the': 6, 'algorithm': 7, 'works': 8, 'as': 9, 'a': 10}


Esta técnica se usa mucho en ML, sin embargo, vamos a ver cómo funciona con nuestro ejemplo:

In [2]:
positive_review = "I read the movie was bad but it was actually awesome"
negative_review = "I read the movie was awesome but it was actually bad"

pos_bag = bag_of_words(positive_review)
neg_bag = bag_of_words(negative_review)

print("Positive:", pos_bag)
print("Negative:", neg_bag)

Positive: {11: 1, 12: 1, 6: 1, 13: 1, 14: 2, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1}
Negative: {11: 1, 12: 1, 6: 1, 13: 1, 14: 2, 19: 1, 16: 1, 17: 1, 18: 1, 15: 1}


Podemos ver que a pesar de tener significados muy diferentes el resultado es el mismo... Vamos a ver si podemos encontrar maneras mejores de condificar nuestras opiniones.

#### Integer Encoding

Integer Encoding es el nombre del méetodo que lo que hace es representar cada palabra o caracter con un entero único y mantener el orden de dichas palabras.


In [3]:
vocabulary = {}  
word_encoding = 1
def one_hot_encoding(text):
  global word_encoding

  words = text.lower().split(" ") 
  encoding = []  

  for word in words:
    if word in vocabulary:
      code = vocabulary[word]  
      encoding.append(code) 
    else:
      vocabulary[word] = word_encoding
      encoding.append(word_encoding)
      word_encoding += 1
  
  return encoding

text = "test sentence to see if the algorithm works as a test see see"
encoding = one_hot_encoding(text)
print(encoding)
print(vocabulary)

positive_review = "I read the movie was bad but it was actually awesome"
negative_review = "I read the movie was awesome but it was actually bad"

pos_encode = one_hot_encoding(positive_review)
neg_encode = one_hot_encoding(negative_review)

print("Positive:", pos_encode)
print("Negative:", neg_encode)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 4, 4]
{'test': 1, 'sentence': 2, 'to': 3, 'see': 4, 'if': 5, 'the': 6, 'algorithm': 7, 'works': 8, 'as': 9, 'a': 10}
Positive: [11, 12, 6, 13, 14, 15, 16, 17, 14, 18, 19]
Negative: [11, 12, 6, 13, 14, 19, 16, 17, 14, 18, 15]


A priori pinta mejor! Este método no pierde el orden de las palabras, sin embargo, a nosotros nos gustaría que cuando codifiquemos palabras, las que sean similares entre sí tengan codificaciones similares y viceversa. Por ejemplo, las palabras feliz y contento deberían tener codificaciones similares, de esta forma nuestra red podrá aprender mucha más información. 

En el método que acabamos de ver la codificación de cada palabra es arbitraria, por tanto el modelo tendrá que hacer mucho trabajo y ver muchos datos para determinar si dos palabras son similares o no. Vamos a ver si podemos mejorarlo aún más!

#### Word Embeddings

El tercer método que vamos a ver se llama Word Embeddings. Este método no sólo mantiene el orden de las palabras si no que además codifica palabras similares con representaciones similares. Codifica a la vez el orden, la frecuencia y el significado de las palabras en cada frase.

El objetivo de los word embeddings es codificar cada palabra como un vector denso (no un 1-hot) de tal forma que palabras similares tengan una distancia entre vectores codificados pequeña.

A diferencia de las técnicas anteriores, estas codificaciones se aprenden a partir de los datos de entrenamiento. Podemos añadir una capa de embedding al principio d enuestro modelo o utilizar capas preentrenadas.

<img src="embeddings.png" style="height: 400px">

### IMBD Sentiment Analysis, caso práctico

Ahora que ya tenemos todos los ingredientes, vamos a ver cómo podemos enfrentarnos a este reto:

El dataset consiste en 25.000 opiniones de películas muy polarizadas (o la película ha gustado mucho o muy poco) para entrenar y otras 25.000 como test. La tarea a resolver es, dada una opinión de test, saber si el usuario tiene una opinión positiva o negativa de la película.

Como siempre, comenzamos importando los módulos y librerías que vamos a necesitar.

In [4]:
import tensorflow as tf

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb


A continuación, vamos a especificar 3 variables que necesitaremos más adelante:
- max_features: Es el número máximo de palabras que tendremos en cuenta de la base de datos, vamos a utilizar 20.000. Esto quiere decir que tan sólo vamos a considerar las 20.000 palabras más comunes de toda la base de datos, el resto serán ignoradas, de forma que el rango de los datos de entrada esté limitado.
- max_len: Es el número de palabras máximo que vamos a permitir por cada opinión. Si una opinión tiene más palabras se truncará y si una opinión tiene menos se añadiran ceros para que todas sean del mismo tamaño.
- batch_size: El tamaño del batch para el entrenamiento

In [5]:
max_features = 20000
maxlen = 150  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

El siguiente paso va a ser cargar los datos. 

Keras tiene acceso al dataset IMDB incluido en sus ejemplos de prueba, la función imdb.load_data() nos permite cargar este dataset en un formato que ya está preparado para ser usado en modelos de inteligencia artificial. Las palabras han sido sustituidas por enteros que indican el orden de frecuencia de cada palabra en el conjunto de los datos. Las frases de cada opinión han sido transformadas a una secuencia de números enteros.


In [6]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

Loading data...


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/opt/anaconda3/envs/mioti_nlp/lib/python3.9/site-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/opt/anaconda3/envs/mioti_nlp/lib/python3.9/site-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or sha

Siguiendo nuestro proceso habitual, procedemos a comprobar que los datos se han bajado de forma adecuada, para ello imprimiremos por pantalla el tamaño de nuestro conjunto de datos.

In [7]:
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [8]:
x_train[1]

[1,
 194,
 1153,
 194,
 8255,
 78,
 228,
 5,
 6,
 1463,
 4369,
 5012,
 134,
 26,
 4,
 715,
 8,
 118,
 1634,
 14,
 394,
 20,
 13,
 119,
 954,
 189,
 102,
 5,
 207,
 110,
 3103,
 21,
 14,
 69,
 188,
 8,
 30,
 23,
 7,
 4,
 249,
 126,
 93,
 4,
 114,
 9,
 2300,
 1523,
 5,
 647,
 4,
 116,
 9,
 35,
 8163,
 4,
 229,
 9,
 340,
 1322,
 4,
 118,
 9,
 4,
 130,
 4901,
 19,
 4,
 1002,
 5,
 89,
 29,
 952,
 46,
 37,
 4,
 455,
 9,
 45,
 43,
 38,
 1543,
 1905,
 398,
 4,
 1649,
 26,
 6853,
 5,
 163,
 11,
 3215,
 10156,
 4,
 1153,
 9,
 194,
 775,
 7,
 8255,
 11596,
 349,
 2637,
 148,
 605,
 15358,
 8003,
 15,
 123,
 125,
 68,
 2,
 6853,
 15,
 349,
 165,
 4362,
 98,
 5,
 4,
 228,
 9,
 43,
 2,
 1157,
 15,
 299,
 120,
 5,
 120,
 174,
 11,
 220,
 175,
 136,
 50,
 9,
 4373,
 228,
 8255,
 5,
 2,
 656,
 245,
 2350,
 5,
 4,
 9837,
 131,
 152,
 491,
 18,
 2,
 32,
 7464,
 1212,
 14,
 9,
 6,
 371,
 78,
 22,
 625,
 64,
 1382,
 9,
 8,
 168,
 145,
 23,
 4,
 1690,
 15,
 16,
 4,
 1355,
 5,
 28,
 6,
 52,
 154,
 462,
 33,


A continuación vamos a resolver uno de los problemas más típicos cuando estamos trabajando con secuencias: la variabilidad en duración de estas.

Para ello vamos a utilizar la función sequence.pad_sequences(input, maxlen) que truncará las secuencias a un largo máximo de maxLen y añadirá ceros a las que sean mas cortas de forma que todas tengan tamaño maxLen al final. El modelo aprenderá que el cero no implica ninguna información pero al tener todas las secuencias con la misma duración el procesado será más rápido y sencillo.

In [9]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

Pad sequences (samples x time)


Por último, procedemos a comprobar que el padding se ha realizado correctamente

In [10]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 150)
x_test shape: (25000, 150)


### Embedding

El primer paso de nuestro modelo va a ser una capa de tipo embedding, esta capa lo que hace es transformar nuestra entrada, que es un entero en el rango de 0 a 19999, a un vector del tamaño que nosotros elijamos (se suele elegir menor al rango de entrada) donde palabras con un significado parecido tendrán vectores cuya distancia entre sí es menor que la de vectores provenientes de palabras con significados distintos.

### Definición del modelo

Sabiendo lo que hace la capa embedding y conociendo los detalles de las redes LSTMs tenemos ya todos los ingredientes para construir nuestro modelo:

In [11]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Build model...


NotImplementedError: Cannot convert a symbolic Tensor (lstm/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

### Compilando el modelo

Ahora que ya tenemos nuestro modelo, vamos a compilarlo como es habitual.

Una de las particularidades de las redes recurrentes es que son mucho más sensibles en su entrenamiento. Por este motivo, el tamaño del batch, la función de coste y el optimizador tienen un impacto mucho mayor que en redes como las que hemos entrenado hasta ahora.

Para comenzar utilizaremos la entropía cruzada como función de coste y el optimizador Adam.

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### Entrenamiento del modelo

Como ya sabemos, el siguiente paso es el entrenamiento del modelo. Si utilizamos el dataset completo el entrenamiento se hace bastante más lento, por este motivo, vamos a utilizar un conjunto de datos de entrenamiento y de test reducidos para poder hacer pruebas de forma más rápida

In [ ]:
print('Train...')
x_train = x_train[1:2500,:]
y_train = y_train[1:2500]

x_test = x_test[1:2500,:]
y_test = y_test[1:2500]

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

### Extra: Predicciones de nuestras opiniones

Ya que tenemos el modelo, vamos a ver cómo podríamos meter nuestras propias opiniones para clasificarlas.

Comenzamos codificando una opinión:

In [ ]:
index = imdb.get_word_index()

def encode_text(text):
  tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [index[word] if word in index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], maxlen)[0]

text = "I loved that movie, it was so awesome"
encoded = encode_text(text)
print(encoded)

Y ahora, vamos a obtener nuestra predicción:

In [ ]:
import numpy as np

def predict(text):
  encoded = encode_text(text)
  pred = np.zeros((1,maxlen))
  pred[0] = encoded
  result = model.predict(pred) 
  print(result[0])


In [ ]:
positive_review = "It is the best movie I have ever seen! It is great, I had an awesome time watching it and I would love to watch it again because I enjoyed every second of it"
predict(positive_review)

negative_review = "I did not like the movie, it really sucked. Seriously, it is bad as hell... I hate when people act so badly and I wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

## Resumen

A continuación tenemos un resumen de lo hecho hasta ahora, con todo el código en un sólo script:

In [ ]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

max_features = 20000
maxlen = 150  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

print('Train...')
x_train = x_train[1:2500,:]
y_train = y_train[1:2500]

x_test = x_test[1:2500,:]
y_test = y_test[1:2500]

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)